# Purpose

```
   Create a databricks notebook (as base python)
   
   Investigate the Databricks DBFS 
   
   Create and read Parquet files, and understand them better 
   
   Create and read JSON files, display etc 
   
   ```

<br>

#### Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data !

In [5]:
# Parquet is a columnar format that is supported by many other data processing systems. 

# Spark SQL provides support for both reading and writing Parquet files that automatically 
# preserves the schema of the original data. 

# When writing Parquet files, all columns are automatically converted to be nullable for compatibility reasons.

# Examine the spark context

In [8]:
# return all of the methods possible under this particular input object
def list_out_object_methods(your_object):
  for method in dir(your_object):
    if not method.startswith("_"):
      print(method)
      
# i.e. i create a spark object, but i want to see all the methods possible on 
# it, use this function.  It helps to list out ALL methods you can enter ! 

In [9]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [10]:
# lets confirm we have a spark context created
# if you know me, u know i always do these first couple steps
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [11]:
list_out_object_methods(spark)

Builder
builder
catalog
conf
createDataFrame
newSession
range
read
readStream
sparkContext
sql
stop
streams
table
udf
version

In [12]:
# list all the methods for the spark context, in case you want to look around
list_out_object_methods(sc)   # then you just enter sc.<the method>

PACKAGE_EXTENSIONS
accumulator
addClusterWideLibraryToPath
addFile
addIsolatedLibraryPath
addPyFile
appName
applicationId
binaryFiles
binaryRecords
broadcast
cancelAllJobs
cancelJobGroup
defaultMinPartitions
defaultParallelism
dump_profiles
emptyRDD
environment
getConf
getLocalProperty
getOrCreate
hadoopFile
hadoopRDD
init_batched_serializer
master
newAPIHadoopFile
newAPIHadoopRDD
parallelize
pickleFile
profiler_collector
pythonExec
pythonVer
range
runJob
sequenceFile
serializer
setCheckpointDir
setJobDescription
setJobGroup
setLocalProperty
setLogLevel
setSystemProperty
show_profiles
sparkHome
sparkUser
startTime
statusTracker
stop
textFile
uiWebUrl
union
version
wholeTextFiles

In [13]:
def gather_up_my_spark_session_infor_for_me(s):
  print("Apache Spark Version . . . . . . . . . . . ",s.version)
  print("Spark master . . . . . . . . . . . . . . . ",s.master)
  print("Core Python Version . . .  . . . . . . . . ",s.pythonVer)
  print("App Name . . . . . . . . . . . . . . . . . ",s.appName)
  print("spark URL  . . . . . . . . . . . . . . . . ",s.uiWebUrl)
  print("Spark Home . . . . . . . . . . . . . . . . ",s.sparkHome)
  print("Spark session started at   . . . . . . . . ",s.startTime)
  print("Spark defaultMinPartitions . . . . . . . . ",s.defaultMinPartitions)
  print("Spark defaultParallelism . . . . . . . . . ",s.defaultParallelism)
  print("Spark hadoopFile . . . . . . . . . . . . . ",s.hadoopFile)
  print("Spark user . . . . . . . . . . . . . . . . ",s.sparkUser)  

In [14]:
gather_up_my_spark_session_infor_for_me(sc)

Apache Spark Version . . . . . . . . . . . 2.4.3
Spark master . . . . . . . . . . . . . . . local[8]
Core Python Version . . . . . . . . . . . 3.5
App Name . . . . . . . . . . . . . . . . . Databricks Shell
spark URL . . . . . . . . . . . . . . . . http://10.172.254.169:48476
Spark Home . . . . . . . . . . . . . . . . /databricks/spark
Spark session started at . . . . . . . . 1566065959439
Spark defaultMinPartitions . . . . . . . . 2
Spark defaultParallelism . . . . . . . . . 8
Spark hadoopFile . . . . . . . . . . . . . <bound method SparkContext.hadoopFile of <SparkContext master=local[8] appName=Databricks Shell>>
Spark user . . . . . . . . . . . . . . . . <bound method SparkContext.sparkUser of <SparkContext master=local[8] appName=Databricks Shell>>

<br>

# Let's look at Parquet Files:

THE standard i think:
*  https://github.com/apache/parquet-format

In [18]:
# side note:  
# upload image to databricks DBFS
# it will show up under DBFS /FileStore/tables/<yourfilename>
# and then from there, IF you want to access the file for HTML image view:
#  /files/svm.jpg
# but for writing, it would be:
#  /FileStore/svm.jpg

Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON.

Parquet stores binary data in a column-oriented way, where the values of each column are organized so that they are all adjacent, enabling better compression. It is especially good for queries which read particular columns from a “wide” (with many columns) table since only needed columns are read and IO is minimized.

<img src ='/files/tables/parquet.gif'>

In [21]:
dbutils.fs.ls("/tmp")

Out[ 10 ]: [FileInfo(path='dbfs:/tmp/flights_delta/', name='flights_delta/', size=0),
 FileInfo(path='dbfs:/tmp/flights_parquet/', name='flights_parquet/', size=0),
 FileInfo(path='dbfs:/tmp/hail.HC0uPxHZRQ1F/', name='hail.HC0uPxHZRQ1F/', size=0),
 FileInfo(path='dbfs:/tmp/hive/', name='hive/', size=0),
 FileInfo(path='dbfs:/tmp/test.json', name='test.json', size=243),
 FileInfo(path='dbfs:/tmp/testParquet/', name='testParquet/', size=0),
 FileInfo(path='dbfs:/tmp/test_dbfs.txt', name='test_dbfs.txt', size=52)]

In [22]:
dbutils.fs.ls("/databricks-datasets")

Out[ 11 ]: [FileInfo(path='dbfs:/databricks-datasets/README.md', name='README.md', size=976),
 FileInfo(path='dbfs:/databricks-datasets/Rdatasets/', name='Rdatasets/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/SPARK_README.md', name='SPARK_README.md', size=3359),
 FileInfo(path='dbfs:/databricks-datasets/adult/', name='adult/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/airlines/', name='airlines/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/amazon/', name='amazon/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/asa/', name='asa/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/atlas_higgs/', name='atlas_higgs/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/bikeSharing/', name='bikeSharing/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/cctvVideos/', name='cctvVideos/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/credit-card-fraud/', name='credit-card-fraud/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/cs100/', name='cs100/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/cs110x/', name='cs110x/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/cs190/', name='cs190/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/data.gov/', name='data.gov/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/', name='definitive-guide/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/flights/', name='flights/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/genomics/', name='genomics/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/hail/', name='hail/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/iot/', name='iot/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/iot-stream/', name='iot-stream/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark/', name='learning-spark/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/', name='learning-spark-v2/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/lending-club-loan-stats/', name='lending-club-loan-stats/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/mnist-digits/', name='mnist-digits/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/news20.binary/', name='news20.binary/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/online_retail/', name='online_retail/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/overlap-join/', name='overlap-join/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/power-plant/', name='power-plant/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/sai-summit-2019-sf/', name='sai-summit-2019-sf/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/sample_logs/', name='sample_logs/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/samples/', name='samples/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/sfo_customer_survey/', name='sfo_customer_survey/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/sms_spam_collection/', name='sms_spam_collection/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/songs/', name='songs/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/structured-streaming/', name='structured-streaming/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/tpch/', name='tpch/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/weather/', name='weather/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/wiki/', name='wiki/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/wikipedia-datasets/', name='wikipedia-datasets/', size=0)]

In [23]:
# path to file (flight data)
path = "/tmp/flights_parquet/Origin=DFW/part-00004-tid-6689744057697969040-c960f890-badf-4220-9b06-f4058ede2b08-105-79.c000.snappy.parquet"

# read and display the parquet file
parquetDF = spark.read.parquet(path)
display(parquetDF.head(7))


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,8,11,1,NA,645,NA,1023,XE,2004,null,NA,158,NA,NA,NA,CLE,1021,NA,NA,1,C,0,NA,NA,NA,NA,NA
2008,6,17,2,653,700,1120,1136,CO,1130,N16650,207,216,177,-16,-7,EWR,1372,8,22,0,null,0,NA,NA,NA,NA,NA
2008,8,14,4,2307,1905,231,2243,XE,2572,N13995,144,158,128,228,242,CLE,1021,5,11,0,null,0,0,0,0,0,228
2008,6,11,3,1917,1918,2016,2028,CO,714,N76354,59,70,42,-12,-1,IAH,224,5,12,0,null,0,NA,NA,NA,NA,NA
2008,8,20,3,1534,1307,1708,1420,XE,2214,N14943,94,73,53,168,147,IAH,224,17,24,0,null,0,0,0,21,0,147
2008,6,12,4,1805,1805,2240,2245,CO,1140,N17620,215,220,193,-5,0,EWR,1372,8,14,0,null,0,NA,NA,NA,NA,NA
2008,8,30,6,1625,1630,1737,1742,XE,2805,N16951,72,72,46,-5,-5,IAH,224,16,10,0,null,0,NA,NA,NA,NA,NA


In [24]:
# side note
#   to completely remove a file:
#     -  dbutils.fs.rm("/tmp/databricks-df-example.parquet", True)

* Good link that talks about why parquet is awesome: https://databricks.com/glossary/what-is-parquet

In [26]:
# You have spark context running, but you also have sqlContext as well...
sqlContext  # confirm it is running...

Out[ 18 ]: <pyspark.sql.context.HiveContext at 0x7fd16d0315f8>

In [27]:
# these are all the things i can 'do' to my DF:
list_out_object_methods(parquetDF)

agg
alias
approxQuantile
cache
checkpoint
coalesce
colRegex
collect
columns
corr
count
cov
createGlobalTempView
createOrReplaceGlobalTempView
createOrReplaceTempView
createTempView
crossJoin
crosstab
cube
describe
distinct
drop
dropDuplicates
drop_duplicates
dropna
dtypes
exceptAll
explain
fillna
filter
first
foreach
foreachPartition
freqItems
groupBy
groupby
head
hint
intersect
intersectAll
isLocal
isStreaming
is_cached
join
limit
localCheckpoint
na
orderBy
persist
printSchema
randomSplit
rdd
registerTempTable
repartition
repartitionByRange
replace
rollup
sample
sampleBy
schema
select
selectExpr
show
sort
sortWithinPartitions
sql_ctx
stat
storageLevel
subtract
summary
take
toDF
toJSON
toLocalIterator
toPandas
transform
union
unionAll
unionByName
unpersist
where
withColumn
withColumnRenamed
withWatermark
write
writeStream

In [28]:
# i want to know how many rows of data are in my parquet file:
parquetDF.count()

Out[ 22 ]: 26299

In [29]:
print(type(parquetDF))  # this is a DataFrame fyi 

<class 'pyspark.sql.dataframe.DataFrame'>

In [30]:
# listing out the columns in the actual dataframe
parquetDF.columns

# very handy to use...

Out[ 25 ]: ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

In [31]:
# i wish to output the column of 'Dest' that are unique entries:
parquetDF.select('Dest').distinct().show()  # like using show, it automatically limits output...

+----+
Dest|
+----+
 MSY|
 SNA|
 BUR|
 GRR|
 GSO|
 OAK|
 FSM|
 MSN|
 DCA|
 MLU|
 CID|
 LEX|
 ORF|
 EVV|
 SAV|
 CMH|
 TYR|
 MOB|
 PNS|
 IAH|
+----+
only showing top 20 rows

In [32]:
parquetDF.schema

Out[ 33 ]: StructType(List(StructField(Year,IntegerType,true),StructField(Month,IntegerType,true),StructField(DayofMonth,IntegerType,true),StructField(DayOfWeek,IntegerType,true),StructField(DepTime,StringType,true),StructField(CRSDepTime,IntegerType,true),StructField(ArrTime,StringType,true),StructField(CRSArrTime,IntegerType,true),StructField(UniqueCarrier,StringType,true),StructField(FlightNum,IntegerType,true),StructField(TailNum,StringType,true),StructField(ActualElapsedTime,StringType,true),StructField(CRSElapsedTime,StringType,true),StructField(AirTime,StringType,true),StructField(ArrDelay,StringType,true),StructField(DepDelay,StringType,true),StructField(Dest,StringType,true),StructField(Distance,IntegerType,true),StructField(TaxiIn,StringType,true),StructField(TaxiOut,StringType,true),StructField(Cancelled,IntegerType,true),StructField(CancellationCode,StringType,true),StructField(Diverted,IntegerType,true),StructField(CarrierDelay,StringType,true),StructField(WeatherDelay,StringType,true),StructField(NASDelay,StringType,true),StructField(SecurityDelay,StringType,true),StructField(LateAircraftDelay,StringType,true)))

In [33]:
list_out_object_methods(parquetDF)

agg
alias
approxQuantile
cache
checkpoint
coalesce
colRegex
collect
columns
corr
count
cov
createGlobalTempView
createOrReplaceGlobalTempView
createOrReplaceTempView
createTempView
crossJoin
crosstab
cube
describe
distinct
drop
dropDuplicates
drop_duplicates
dropna
dtypes
exceptAll
explain
fillna
filter
first
foreach
foreachPartition
freqItems
groupBy
groupby
head
hint
intersect
intersectAll
isLocal
isStreaming
is_cached
join
limit
localCheckpoint
na
orderBy
persist
printSchema
randomSplit
rdd
registerTempTable
repartition
repartitionByRange
replace
rollup
sample
sampleBy
schema
select
selectExpr
show
sort
sortWithinPartitions
sql_ctx
stat
storageLevel
subtract
summary
take
toDF
toJSON
toLocalIterator
toPandas
transform
union
unionAll
unionByName
unpersist
where
withColumn
withColumnRenamed
withWatermark
write
writeStream

```

All the methods on the DF that are possible:

agg
alias
approxQuantile
cache
checkpoint
coalesce
colRegex
collect
columns
corr
count
cov
createGlobalTempView
createOrReplaceGlobalTempView
createOrReplaceTempView
createTempView
crossJoin
crosstab
cube
describe
distinct
drop
dropDuplicates
drop_duplicates
dropna
dtypes
exceptAll
explain
fillna
filter
first
foreach
foreachPartition
freqItems
groupBy
groupby
head
hint
intersect
intersectAll
isLocal
isStreaming
is_cached
join
limit
localCheckpoint
na
orderBy
persist
printSchema
randomSplit
rdd
registerTempTable
repartition
repartitionByRange
replace
rollup
sample
sampleBy
schema
select
selectExpr
show
sort
sortWithinPartitions
sql_ctx
stat
storageLevel
subtract
summary
take
toDF
toJSON
toLocalIterator
toPandas
transform
union
unionAll
unionByName
unpersist
where
withColumn
withColumnRenamed
withWatermark
write
writeStream
```

##### Use SQL direct commands, which i really like:

*Create a temporary table and then we can query it*

<br>
<br>

# Parquet Example #2

In [39]:
# lets look at a large file
# File uploaded to /FileStore/tables/userdata1.parquet

A good source of some parquet files:
* https://github.com/Teradata/kylo/tree/master/samples/sample-data/parquet

In [41]:
dbutils.fs.ls("dbfs:/FileStore/tables")    # you can see your file there

Out[ 41 ]: [FileInfo(path='dbfs:/FileStore/tables/BreseeMSDS7330Unit11HW.docx', name='BreseeMSDS7330Unit11HW.docx', size=1807476),
 FileInfo(path='dbfs:/FileStore/tables/Cbp/', name='Cbp/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/README.md', name='README.md', size=2033),
 FileInfo(path='dbfs:/FileStore/tables/SMU/', name='SMU/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/beer_data.csv', name='beer_data.csv', size=3143),
 FileInfo(path='dbfs:/FileStore/tables/boston_train.csv', name='boston_train.csv', size=24462),
 FileInfo(path='dbfs:/FileStore/tables/iris.csv', name='iris.csv', size=3858),
 FileInfo(path='dbfs:/FileStore/tables/json.json', name='json.json', size=243),
 FileInfo(path='dbfs:/FileStore/tables/mushrooms.csv', name='mushrooms.csv', size=382127),
 FileInfo(path='dbfs:/FileStore/tables/parquet.gif', name='parquet.gif', size=43589),
 FileInfo(path='dbfs:/FileStore/tables/pledge_of_allegiance.txt', name='pledge_of_allegiance.txt', size=174),
 FileInfo(path='dbfs:/FileStore/tables/raw_csv_diamonds.csv', name='raw_csv_diamonds.csv', size=2772143),
 FileInfo(path='dbfs:/FileStore/tables/tom_diamonds_data.csv', name='tom_diamonds_data.csv', size=2772143),
 FileInfo(path='dbfs:/FileStore/tables/userdata1.parquet', name='userdata1.parquet', size=113629)]

In [42]:
dataP = spark.read.parquet("dbfs:/FileStore/tables/userdata1.parquet")

# see how it infers the schema ? 



# -> Number of rows in the file: 1000
# -> Column details:
# column#		column_name		hive_datatype
# =====================================================
# 1		registration_dttm 	    timestamp
# 2		id 			            int
# 3		first_name 		        string
# 4		last_name 		        string
# 5		email 			        string
# 6		gender 			        string
# 7		ip_address 		        string
# 8		cc 			            string
# 9		country 		        string
# 10	birthdate 		        string
# 11	salary 			        double
# 12	title 			        string
# 13	comments 		        string



In [43]:
dataP.show(10)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+--------------------+----------+---------+--------------------+--------+
 registration_dttm| id|first_name|last_name| email|gender| ip_address| cc| country| birthdate| salary| title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+--------------------+----------+---------+--------------------+--------+
2016-02-03 07:55:29| 1| Amanda| Jordan| ajordan0@com.com|Female| 1.197.201.2|6759521864920116| Indonesia| 3/8/1971| 49756.53| Internal Auditor| 1E+02|
2016-02-03 17:04:03| 2| Albert| Freeman| afreeman1@is.gd| Male|218.111.175.34| | Canada| 1/16/1968|150280.17| Accountant IV| |
2016-02-03 01:09:31| 3| Evelyn| Morgan|emorgan2@altervis...|Female| 7.161.136.94|6767119071901597| Russia| 2/1/1960|144972.51| Structural Engineer| |
2016-02-03 00:36:21| 4| Denise| Riley| driley3@gmpg.org|Female| 140.35.109.83|3576031598965625| China| 4/8/1997| 90263.05|Senior Cost Accou...| |
2016-02-03 05:05:31| 5| Carlos| Burns|cburns4@miitbeian...| |169.113.235.40|5602256255204850| South Africa| | null| | |
2016-02-03 07:22:34| 6| Kathryn| White| kwhite5@google.com|Female|195.131.81.179|3583136326049310| Indonesia| 2/25/1983| 69227.11| Account Executive| |
2016-02-03 08:33:08| 7| Samuel| Holmes|sholmes6@foxnews.com| Male|232.234.81.197|3582641366974690| Portugal|12/18/1987| 14247.62|Senior Financial ...| |
2016-02-03 06:47:06| 8| Harry| Howell| hhowell7@eepurl.com| Male| 91.235.51.73| |Bosnia and Herzeg...| 3/1/1962|186469.43| Web Developer IV| |
2016-02-03 03:52:53| 9| Jose| Foster| jfoster8@yelp.com| Male| 132.31.53.61| | South Korea| 3/27/1992|231067.84|Software Test Eng...| 1E+02|
2016-02-03 18:29:47| 10| Emily| Stewart|estewart9@opensou...|Female|143.28.251.245|3574254110301671| Nigeria| 1/28/1997| 27234.28| Health Coach IV| |
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+--------------------+----------+---------+--------------------+--------+
only showing top 10 rows

In [44]:
# I can poll the schema directly
dataP.schema

Out[ 47 ]: StructType(List(StructField(registration_dttm,TimestampType,true),StructField(id,IntegerType,true),StructField(first_name,StringType,true),StructField(last_name,StringType,true),StructField(email,StringType,true),StructField(gender,StringType,true),StructField(ip_address,StringType,true),StructField(cc,StringType,true),StructField(country,StringType,true),StructField(birthdate,StringType,true),StructField(salary,DoubleType,true),StructField(title,StringType,true),StructField(comments,StringType,true)))

In [45]:
# feeling lazy ? 

# upload parquet file, read it into dataframe in python, and then use 
# databrick's export 'Download csv' buttom (far right downward arrow) to
# download and view the csv form on your laptop...

In [46]:
dataP.count()  # 1,000 rows

Out[ 50 ]: 1000

<br>

# Let's examine JSON

In [49]:
# i manually uploaded this file called json.json
# sidenote: If your cluster is running Databricks Runtime 4.0 and above, you can read JSON 
# files in single-line or multi-line mode. In single-line mode, a file can be split into 
# many parts and read in parallel.

# lets confirm i can see it in my storage: 
display(dbutils.fs.ls("dbfs:/FileStore/tables/json.json"))

path,name,size
dbfs:/FileStore/tables/json.json,json.json,243


In [50]:
randomDF = spark.read.json("dbfs:/FileStore/tables/json.json")

In [51]:
# this is my json.json file i uploaded:
#  {"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
#  {"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
#  {"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key":extra_value3"}}


# but the cool part is that Spark infers the schema automatically
randomDF.printSchema

Out[ 55 ]: <bound method DataFrame.printSchema of DataFrame[array: array<bigint>, dict: struct<extra_key:string,key:string>, int: bigint, string: string]>

In [52]:
randomDF.show()

+---------+--------------------+---+-------+
 array| dict|int| string|
+---------+--------------------+---+-------+
[1, 2, 3]| [, value1]| 1|string1|
[2, 4, 6]| [, value2]| 2|string2|
[3, 6, 9]|[extra_value3, va...| 3|string3|
+---------+--------------------+---+-------+

In [53]:
display(randomDF)

array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


<br>

## Another JSON example:

In [56]:
import json
import requests

# You’ll need to make an API request to the JSONPlaceholder service, so just use the requests 
# package to do the heavy lifting.
response = requests.get("https://jsonplaceholder.typicode.com/todos")
todos = json.loads(response.text)


In [57]:
response.json()

Out[ 62 ]: [{'completed': False, 'id': 1, 'title': 'delectus aut autem', 'userId': 1},
 {'completed': False,
 'id': 2,
 'title': 'quis ut nam facilis et officia qui',
 'userId': 1},
 {'completed': False, 'id': 3, 'title': 'fugiat veniam minus', 'userId': 1},
 {'completed': True, 'id': 4, 'title': 'et porro tempora', 'userId': 1},
 {'completed': False,
 'id': 5,
 'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum',
 'userId': 1},
 {'completed': False,
 'id': 6,
 'title': 'qui ullam ratione quibusdam voluptatem quia omnis',
 'userId': 1},
 {'completed': False,
 'id': 7,
 'title': 'illo expedita consequatur quia in',
 'userId': 1},
 {'completed': True,
 'id': 8,
 'title': 'quo adipisci enim quam ut ab',
 'userId': 1},
 {'completed': False,
 'id': 9,
 'title': 'molestiae perspiciatis ipsa',
 'userId': 1},
 {'completed': True,
 'id': 10,
 'title': 'illo est ratione doloremque quia maiores aut',
 'userId': 1},
 {'completed': True,
 'id': 11,
 'title': 'vero rerum temporibus dolor',
 'userId': 1},
 {'completed': True,
 'id': 12,
 'title': 'ipsa repellendus fugit nisi',
 'userId': 1},
 {'completed': False, 'id': 13, 'title': 'et doloremque nulla', 'userId': 1},
 {'completed': True,
 'id': 14,
 'title': 'repellendus sunt dolores architecto voluptatum',
 'userId': 1},
 {'completed': True,
 'id': 15,
 'title': 'ab voluptatum amet voluptas',
 'userId': 1},
 {'completed': True,
 'id': 16,
 'title': 'accusamus eos facilis sint et aut voluptatem',
 'userId': 1},
 {'completed': True,
 'id': 17,
 'title': 'quo laboriosam deleniti aut qui',
 'userId': 1},
 {'completed': False,
 'id': 18,
 'title': 'dolorum est consequatur ea mollitia in culpa',
 'userId': 1},
 {'completed': True,
 'id': 19,
 'title': 'molestiae ipsa aut voluptatibus pariatur dolor nihil',
 'userId': 1},
 {'completed': True,
 'id': 20,
 'title': 'ullam nobis libero sapiente ad optio sint',
 'userId': 1},
 {'completed': False,
 'id': 21,
 'title': 'suscipit repellat esse quibusdam voluptatem incidunt',
 'userId': 2},
 {'completed': True,
 'id': 22,
 'title': 'distinctio vitae autem nihil ut molestias quo',
 'userId': 2},
 {'completed': False,
 'id': 23,
 'title': 'et itaque necessitatibus maxime molestiae qui quas velit',
 'userId': 2},
 {'completed': False,
 'id': 24,
 'title': 'adipisci non ad dicta qui amet quaerat doloribus ea',
 'userId': 2},
 {'completed': True,
 'id': 25,
 'title': 'voluptas quo tenetur perspiciatis explicabo natus',
 'userId': 2},
 {'completed': True, 'id': 26, 'title': 'aliquam aut quasi', 'userId': 2},
 {'completed': True,
 'id': 27,
 'title': 'veritatis pariatur delectus',
 'userId': 2},
 {'completed': False,
 'id': 28,
 'title': 'nesciunt totam sit blanditiis sit',
 'userId': 2},
 {'completed': False, 'id': 29, 'title': 'laborum aut in quam', 'userId': 2},
 {'completed': True,
 'id': 30,
 'title': 'nemo perspiciatis repellat ut dolor libero commodi blanditiis omnis',
 'userId': 2},
 {'completed': False,
 'id': 31,
 'title': 'repudiandae totam in est sint facere fuga',
 'userId': 2},
 {'completed': False,
 'id': 32,
 'title': 'earum doloribus ea doloremque quis',
 'userId': 2},
 {'completed': False, 'id': 33, 'title': 'sint sit aut vero', 'userId': 2},
 {'completed': False,
 'id': 34,
 'title': 'porro aut necessitatibus eaque distinctio',
 'userId': 2},
 {'completed': True,
 'id': 35,
 'title': 'repellendus veritatis molestias dicta incidunt',
 'userId': 2},
 {'completed': True,
 'id': 36,
 'title': 'excepturi deleniti adipisci voluptatem et neque optio illum ad',
 'userId': 2},
 {'completed': False, 'id': 37, 'title': 'sunt cum tempora', 'userId': 2},
 {'completed': False, 'id': 38, 'title': 'totam quia non', 'userId': 2},
 {'completed': False,
 'id': 39,
 'title': 'doloremque quibusdam asperiores libero corrupti illum qui omnis',
 'userId': 2},
 {'completed': True,
 'id': 40,
 'title': 'totam atque quo nesciunt',
 'userId': 2},
 {'completed': False,
 'id': 41,
 'title': 'aliquid amet impedit consequatur aspernatur placeat eaque

In [58]:
todos == response.json()

Out[ 63 ]: True

In [59]:
type(todos)

Out[ 64 ]: list

In [60]:
todos[:4]

Out[ 65 ]: [{'completed': False, 'id': 1, 'title': 'delectus aut autem', 'userId': 1},
 {'completed': False,
 'id': 2,
 'title': 'quis ut nam facilis et officia qui',
 'userId': 1},
 {'completed': False, 'id': 3, 'title': 'fugiat veniam minus', 'userId': 1},
 {'completed': True, 'id': 4, 'title': 'et porro tempora', 'userId': 1}]

In [61]:
display(todos)

completed,id,title,userId
false,1,delectus aut autem,1
false,2,quis ut nam facilis et officia qui,1
false,3,fugiat veniam minus,1
true,4,et porro tempora,1
false,5,laboriosam mollitia et enim quasi adipisci quia provident illum,1
false,6,qui ullam ratione quibusdam voluptatem quia omnis,1
false,7,illo expedita consequatur quia in,1
true,8,quo adipisci enim quam ut ab,1
false,9,molestiae perspiciatis ipsa,1
true,10,illo est ratione doloremque quia maiores aut,1


<br>

## another JSON example

In [64]:
dbutils.fs.put("/tmp/test.json", """
{"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
{"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
{"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key": "extra_value3"}}
""", True)

Wrote 243 bytes.
 Out[ 67 ]: True

In [65]:
testJsonData = sqlContext.read.json("/tmp/test.json")

display(testJsonData)

array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


In [66]:
%sql 
    CREATE TEMPORARY TABLE jsonTable
    USING json
    OPTIONS (
      path "/tmp/test.json"
    )

In [67]:
%sql SELECT * FROM jsonTable


array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


<br>

# Databricks Delta Examination

>  *lets look at a deeper examination of parquet and delta, and if we can make some improvements*

In [71]:
# note in the real world, using non Databricks, you can just print out as well

flights = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("/databricks-datasets/asa/airlines/2008.csv")

# see how the schema is inferred ???

In [72]:
# write a parquet-based table using this flights data 
# i specifically partition by ORIGIN
# once i do this, it will spread files around, and breakout by folders of Origin

# flights.write.format("parquet").mode("overwrite").partitionBy("Origin").save("/tmp/flights_parquet_2")

In [73]:
# your output files
#   display(dbutils.fs.ls("dbfs:/tmp/flights_parquet"))  # you can see your file there

In [74]:
# 8 core files per origin, example shown:
#  display(dbutils.fs.ls("dbfs:/tmp/flights_parquet/Origin=DFW/"))  # you can see your file there

In [75]:
# seeing display command in action, side note:
# from pyspark.sql.functions import avg
# diamonds_df = spark.read.csv("/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv", header="true", inferSchema="true")
# display(diamonds_df.select("color","price").groupBy("color").agg(avg("price")))

In [76]:
# lets look at one single subFile

curiousDF = sqlContext.read.parquet("dbfs:/tmp/flights_parquet/Origin=DFW/part-00004-tid-6689744057697969040-c960f890-badf-4220-9b06-f4058ede2b08-105-79.c000.snappy.parquet")

print("This individual dataset has the following number of rows: ", curiousDF.count())

This individual dataset has the following number of rows: 26299

Once step 1 completes, the "flights" table contains details of US flights for a year.

Next in Step 2, we run a query that get top 20 cities with highest monthly total flights on first day of week.

In [78]:
# so when you open up the tmp folder, you see a few hundred files distributed by 
# origin id of like DFW airport.  

# eight files per original

# ex: /tmp/flights_parquet/Origin=DFW/part-00000-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-421-79.c000.snappy.parquet

#### step 2

In [80]:
# lets run a query:

from pyspark.sql.functions import count

# this has got to be a huge file, its consolidating everything 
flights_parquet = spark.read.format("parquet").load("/tmp/flights_parquet")

# query the huge file, breakout by day of week 1, groupby, and count totals
display(flights_parquet.filter("DayOfWeek=1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

# this took 1.72 minutes ! 
# ran it again, and then it 2.46 minutes


Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


In [81]:
print(type(flights_parquet))

<class 'pyspark.sql.dataframe.DataFrame'>

In [82]:
flights_parquet.count()
# thats over 7 million rows dude, pretty hard core for a quick check ...

Out[ 80 ]: 7009728

In [83]:
# very very good command for seeing the schema you have in place for the DF
#############################a
flights_parquet.printSchema()
#############################

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: string (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: string (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: integer (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: string (nullable = true)
-- CRSElapsedTime: string (nullable = true)
-- AirTime: string (nullable = true)
-- ArrDelay: string (nullable = true)
-- DepDelay: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: string (nullable = true)
-- TaxiOut: string (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: string (nullable = true)
-- WeatherDelay: string (nullable = true)
-- NASDelay: string (nullable = true)
-- SecurityDelay: string (nullable = true)
-- LateAircraftDelay: string (nullable = true)
-- Origin: string (nullable = true)

In [84]:
flights_parquet.columns

Out[ 82 ]: ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'Origin']

#### step 3

Once step 2 completes, you can observe the latency with the standard "flights_parquet" table.

In step 3 and step 4, we do the same with a Databricks Delta table. This time, before running the query, we run the OPTIMIZE command with ZORDER to ensure data is optimized for faster retrieval.

In [87]:

# Step 3: Write a Databricks Delta based table using flights data
flights.write.format("delta").mode("overwrite").partitionBy("Origin").save("/tmp/flights_delta")


In [88]:

# Step 3 Continued: OPTIMIZE the Databricks Delta table

display(spark.sql("DROP TABLE  IF EXISTS flights"))

display(spark.sql("CREATE TABLE flights USING DELTA LOCATION '/tmp/flights_delta'"))
                  
display(spark.sql("OPTIMIZE flights ZORDER BY (DayofWeek)"))


In [89]:
# Step 4 : Rerun the query from Step 2 and observe the latency

flights_delta = spark.read.format("delta").load("/tmp/flights_delta")

display(flights_delta.filter("DayOfWeek=1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

# its faster now 


In [90]:

#  HMMMMMMMM

#  it went from taking:  1.72 minutes (and then 2.46 mins)
#  to taking about    :  42.35 seconds
#  about 2.43X + faster...


<br>

## Appendix:  Access DBFS and looking around at functionality

In [93]:
display(dbutils.fs)

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In [94]:

#  write a temp file to DBFS with python i/o apis
#  then print it out 
#  all python based (base notebook is python when created)

#write a file to DBFS using python i/o apis
with open("/dbfs/tmp/test_dbfs.txt", 'w') as f:
  f.write("Apache Spark is the bomb\n")
  f.write("Tom Bresee was the bomb\n")
  f.write("Now Databricks is the bomb\n")
  f.write("Goodbye.")

# read the file
with open("/dbfs/tmp/test_dbfs.txt", "r") as f_read:
  for line in f_read:
    print (line)
    
# df.write.text("/tmp/foo.txt")


Apache Spark is the bomb

Tom Bresee was the bomb

Now Databricks is the bomb

Goodbye.

In [95]:
# When you’re using Spark APIs, you reference files with "/mnt/training/file.csv" or 
# "dbfs:/mnt/training/file.csv". If you’re using local file APIs, you must provide 
# the path under /dbfs, for example: "/dbfs/mnt/training/file.csv". You cannot use 
# a path under dbfs when using Spark APIs.

In [96]:
%scala
// Now read the file you just created with the Scala programming language...

import scala.io.Source

val filename = "/dbfs/tmp/test_dbfs.txt"  // what you just created 

for (line <- Source.fromFile(filename).getLines()) {
  println(line)
}

Apache Spark is awesome!
This is fun
End of example!
import scala.io.Source
filename: String = /dbfs/tmp/test_dbfs.txt

In [97]:
# create a directory called foobar
dbutils.fs.mkdirs("/foobar/")

# now if you go to your main menu, and then click Import and Explore Data,
# you will see this new folder you just created ! 

Out[ 87 ]: True

In [98]:
# put the verbage into the file 
dbutils.fs.put("/foobar/baz.txt", "Hello, World!")

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2053271440068343> in <module> () 
 1 # put the verbage into the file 
 ----> 2 dbutils . fs . put ( "/foobar/baz.txt" , "Hello, World!" ) 

 /local_disk0/tmp/1566065976228-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 306 exc . __context__ = None 
 307 exc . __cause__ = None 
 --> 308 raise exc 
 309 return f_with_exception_handling 
 310 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.put.
: java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 10800); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 10800)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$put$1.apply(DBUtilsCore.scala:222)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$put$1.apply(DBUtilsCore.scala:217)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.put(DBUtilsCore.scala:217)
	at com.databricks.backend.daemon.dbutils.FSUtils.put(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 10800)
	at com.databricks.api.rpc.ScalaProtoRpcSerializer.deserializeException(ScalaProtoRpcSerializer.scala:229)
	at com.databricks.api.rpc.ScalaProtoRpcSerializer.deserializeException(ScalaProtoRpcSerializer.scala:220)
	at com.databricks.rpc.JettyClient.$anonfun$getResponseFromExchan

In [99]:
# thats awesome, i blew out the free edition in the first two days... 

In [100]:
# list out all the folders you have in DBFS ! 
# this is really a 'dir' level command ! 

display(dbutils.fs.ls("dbfs:/"))

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/cbp.vds/,cbp.vds/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/foobar/,foobar/,0
dbfs:/kdd/,kdd/,0
dbfs:/local_disk0/,local_disk0/,0
dbfs:/ml/,ml/,0


In [101]:
# lets look around into the Parquet folder you created earlier ! ! ! 

display(dbutils.fs.ls("dbfs:/tmp/testParquet"))

path,name,size
dbfs:/tmp/testParquet/_committed_354797835800861187,_committed_354797835800861187,1205
dbfs:/tmp/testParquet/_committed_3720738684160984522,_committed_3720738684160984522,1217
dbfs:/tmp/testParquet/_committed_4775365216835712372,_committed_4775365216835712372,1216
dbfs:/tmp/testParquet/_committed_5658643342479558245,_committed_5658643342479558245,1223
dbfs:/tmp/testParquet/_committed_6765437422472151022,_committed_6765437422472151022,1211
dbfs:/tmp/testParquet/_committed_81539086910767789,_committed_81539086910767789,1211
dbfs:/tmp/testParquet/_started_1885843275169497,_started_1885843275169497,0
dbfs:/tmp/testParquet/_started_354797835800861187,_started_354797835800861187,0
dbfs:/tmp/testParquet/_started_3720738684160984522,_started_3720738684160984522,0
dbfs:/tmp/testParquet/_started_5539009013964348726,_started_5539009013964348726,0


In [102]:
# wanna know something odd ?   
# you CANT put comments into the same cell as a filesystem magic command ! 
# what the what, it will erorr out, so it has to be a stand alone cell ! 

In [103]:
# try:
#     import jira.client
#     JIRA_IMPORTED = True

# except ImportError:
#     JIRA_IMPORTED = False